In [ ]:
from spks import *
# from pathlib import Path
folder = Path('/scratch/ks25_sorting_20230905_131941_tr9v')

with open(folder/'params.py','r') as f:
    params = f.read()
params = params.split('\n')
for i,p in enumerate(params):
    if p.startswith('dat_path'):
        params[i] = "dat_path = 'filtered_recording.ap.bin'"
with open(folder/'params.py','w') as f:
    f.write("\n".join(params))


In [ ]:
%matplotlib widget

from spks import *

folder = Path('/scratch/ks25_sorting_20230905_131941_tr9v')
sp = Clusters(folder, get_waveforms=True)


In [ ]:
sp & 'KSLabel = good' & 'shank = 1'

In [ ]:
[sp.cluster_groups.KSLabel == 'good']

In [ ]:
from spks import *

sp = Clusters(folder='/home/data/JC131/20230909_175248/kilosort25/imec0/')
sessions = ['/home/data/JC131/20230909_175248','/home/data/JC131/20230909_194417']

# Load the log and all for the stimulus.
# med filter to get rid of noise (this takes a while)
(onsets,offsets),(sync,meta),sessiondata = load_ni_sync_data(sessions[0])
from scipy.ndimage import median_filter
photodiode = np.array(median_filter(sync[:,0].flatten(),61)>12500,dtype = int)
a = np.where(np.diff(photodiode) != 0)[0]
log = pd.read_csv(list(Path(sessions[0]).glob('adaptation*/*.csv'))[0])
# get the sync interpolation function
from scipy.interpolate import interp1d
interpfunction = interp1d(np.array(onsets[7]),sessiondata[0]['file0_sync_onsets'][6],fill_value='extrapolate')
log['photodiode_samples'] = interpfunction(a).astype(np.uint64)

from btss import read_visual_protocol

options,params = read_visual_protocol(list(Path(sessions[0]).glob('adaptation*/*.visprot'))[0])

params = params[params.contrast>0]
stim_idx = np.hstack([1,params.iloc[np.where(np.diff(params.mu)!=0)[0]].index])


srate = sessiondata[0]['sampling_rate']
ksgood = []
for i in tqdm(sp.cluster_groups[sp.cluster_groups['KSLabel'] == 'good'].cluster_id.values, desc = 'Collecting "good" units'):
    ii = np.where(sp.unique_clusters==i)[0]
    ksgood.append(sp[ii]/srate)
    
# aaa = np.where(np.diff(photodiode) != 0)[0]
events = np.array(log.photodiode_samples)/srate

ts = get_triggered_spikes(ksgood,events, 0, 0.3)

In [ ]:

%matplotlib widget
plot_raster(ts[64][:1000],offset = 0,colors = ['r']*500 +['k']*500)
plot_raster(ts[64][1000:],offset = 1000,colors = 'b')


In [ ]:
import matplotlib.style as mplstyle
mplstyle.use('fast')

# icluster = 20
# wave = sp.get_cluster_waveforms(icluster,n_waveforms=50)
# plot_footprints(wave,sp.channel_positions,gain=[10,0.05],lw = 0.1,color='k');
# plot_footprints(sp.cluster_waveforms_mean[sp.unique_clusters==icluster],sp.channel_positions,gain=[10,0.05],color='r',lw=1);
sp.cluster_waveforms_std[1].shape

In [ ]:
icluster = 0


interactive_cluster_waveforms(sp)

In [ ]:
sp.unique_clusters == 96

In [ ]:
plot_func(234)

In [ ]:
waveforms = sp.cluster_waveforms_mean
channel_positions = sp.channel_positions

peak_to_peak = (waveforms.max(axis = 1) - waveforms.min(axis = 1))
# the amplitude of each waveform is the max of the peak difference for all channels
amplitude = np.abs(peak_to_peak).max(axis=1)
# compute the center of mass (X,Y) of the waveforms
centerofmass = [peak_to_peak*pos for pos in channel_positions.T]
centerofmass = np.vstack([np.sum(t,axis =1 )/np.sum(peak_to_peak,axis = 1) 
                                    for t in centerofmass]).T
# the peak channel is the absolute max of the peak_to_peak
peak_channels = np.argmax(np.abs(peak_to_peak),axis = 1)
print(peak_to_peak.shape,waveforms.shape,peak_channels.shape)

In [ ]:
plt.figure()
plt.plot(peak_to_peak[234])
peak_channels = np.argmax(np.max(np.abs(waveforms),axis=1),axis=1)
# peak_channels = np.argmax(np.abs(peak_to_peak),axis = 1)
plt.plot(peak_channels[234],peak_to_peak[234][peak_channels[234]],'ro')
channel_positions[peak_channels[234]]

In [ ]:
plt.figure()
maxim = waveforms.max(axis=1)
pc2 = np.argmax(np.abs(waveforms).max(axis=1),axis=1)
imaxim = waveforms.argmax(axis=1)
minim = waveforms.min(axis=1)
iminim = waveforms.argmin(axis=1)
icluster = 144
# mm = maxim[icluster][iw]-minim[icluster][iw]
pc = np.argmax(maxim-minim,axis=1)
for iw in range(waveforms[icluster].shape[1]):
    x,y = channel_positions[iw]
    plt.plot(x+0.4*np.arange(waveforms.shape[1]),waveforms[icluster][:,iw]*0.1+y,lw=0.5,color='k');
    plt.plot(x+0.4*imaxim[icluster][iw],maxim[icluster][iw]*0.1+y,'ro',markersize = 1);
    plt.plot(x+0.4*iminim[icluster][iw],minim[icluster][iw]*0.1+y,'bo',markersize = 1);
    col='k'
    if iw == pc2[icluster]:
        col = 'r'
    if iw == principal_channel[icluster]:
        col = 'g'

    plt.text(x+0.4*30,y,'{0:.0f}'.format(maxim[icluster][iw]-minim[icluster][iw]),fontsize=5,color = col);

In [ ]:
imaxim.shape

In [ ]:

# folder
dat2 = map_binary(folder/'filtered_recording.ap.bin',meta['nchannels'])
meta = load_dict_from_h5(folder/'filtered_recording.ap.metadata.hdf')
meta['file_offsets'],dat.file_sample_offsets,dat.shape,dat2.shape

In [ ]:
# Extract waveforms
folder = Path('/scratch/ks25_sorting_20230905_131941_tr9v')
sessionfiles = ['/home/data/JC131/20230901_113844/ephys_g0/ephys_g0_imec0/ephys_g0_t0.imec0.ap.bin',
                '/home/data/JC131/20230901_115632/ephys_g1/ephys_g1_imec0/ephys_g1_t0.imec0.ap.bin']

dat = RawRecording(sessionfiles,return_preprocessed = False)

meta = load_dict_from_h5(folder/'filtered_recording.ap.metadata.hdf')
dat2 = map_binary(folder/'filtered_recording.ap.bin',meta['nchannels'])

sp = Clusters(folder)

mwaves = extract_waveform_set(spike_times = sp,
                              data = dat2,
                              chmap = dat.channel_info.channel_idx.values,
                              max_n_spikes=1000,
                              chunksize=10)
waveforms = {}
for iclu,w in zip(sp.unique_clusters,mwaves):
    waveforms[iclu] = w
save_dict_to_h5(folder/'cluster_waveforms.hdf',waveforms)


In [ ]:


#waveforms = load_dict_from_h5(folder/'cluster_waveforms.hdf')

import h5py as h5
waveforms_file = h5.File(folder/'cluster_waveforms.hdf','r')

In [ ]:
# %matplotlib widget 
import pylab as plt
from spks import *

plt.figure()


iclus = 200
for mw in mwaves[iclus][:20]:
    plot_footprints(waves = waveforms,
                    channel_xy = np.stack(dat.channel_info.channel_coord.values), gain=[15,0.07],lw = 0.1);

plot_footprints(waves = mwaves[iclus].mean(axis=0),
                    channel_xy = np.stack(dat.channel_info.channel_coord.values), gain=[15,0.07],lw = 1,color = 'r');

In [ ]:

pos, peak = waveforms_position(sp.templates_raw,sp.channel_positions)
peak_to_peak = (sp.templates_raw.max(axis = 1) - sp.templates_raw.min(axis = 1)).max(axis=1)

###
import pylab as plt
%matplotlib widget
plt.figure()
plt.plot(sp.channel_positions[:,0],sp.channel_positions[:,1],'o',color='lightgray')
plt.scatter(pos[20,0],pos[20,1],30,peak_to_peak[20],alpha = 0.5,cmap='hot')
plt.colorbar()
plt.plot(sp.channel_positions[peak[20],0],sp.channel_positions[peak[20],1],'x')
from spks.viz import plot_footprints
plot_footprints(sp.templates_raw[20],sp.channel_positions,gain=[5,0.3]);
plt.show()

In [ ]:
sp['unit selection','probe','shank','unit']

In [ ]:
spike_clusters = sp.clusters
spike_times = sp.spike_times


In [ ]:
sp.templates_raw.shape

In [ ]:


get_overlapping_spikes_indices(sp.spike_times,sp.clusters, sp.templates_raw, sp.channel_positions)

In [ ]:
list(folder.glob("*"))

In [ ]:
from spks.sorting import run_ks25
sessionfiles = ['/home/data/JC131/20230901_113844/ephys_g0/ephys_g0_imec0/ephys_g0_t0.imec0.ap.bin',
                '/home/data/JC131/20230901_115632/ephys_g1/ephys_g1_imec0/ephys_g1_t0.imec0.ap.bin']

run_ks25(sessionfiles)

with open(folder/'params.py','r') as f:
    params = f.read()
params = params.split('\n')
for i,p in enumerate(params):
    if p.startswith('dat_path'):
        params[i] = "dat_path = 'filtered_recording.ap.bin'"
with open(folder/'params.py','w') as f:
    f.write("\n".join(params))